# <center> Codebase for creating Monkeypox Knowledge Graph

### If you are using mybinder, from the list of headers above (File, Edit, View...), go to Help -> Launch Classic Notebook for optimal experience. Read more at https://github.com/Fraunhofer-ITMP/mpox-kg/blob/main/README.md

### System details

In [ ]:
import getpass
import sys
import time
import json

In [ ]:
getpass.getuser()

In [ ]:
sys.version

In [ ]:
time.asctime()

### Importing packages

In [ ]:
#pip install -r requirements.txt

In [ ]:
import pybel
from pybel.io.jupyter import to_jupyter
from utils import *
import matplotlib.pyplot as plt

# The following codes have to be executed line by line to generate the final KG. Alternatively, you can use the cached file of the final KG. Press Ctrl+F and type 'Final KG'

# Drugs against monkeypox 

This data was collected from ChEMBL database

In [ ]:
mpox_chembl = pd.read_csv(
    'data/chembl/chembl_mpox_actives_new.csv',
    sep=';',
    usecols=[
        'Molecule ChEMBL ID',
        'pChEMBL Value'
    ])
mpox_chembl= mpox_chembl[mpox_chembl['pChEMBL Value'] >= 6]
mpox_chembl = set(mpox_chembl['Molecule ChEMBL ID'])

In [ ]:
mpox_chembl

# Extracting data from PubChem

Following line of codes extract information from [PubChem](https://pubchem.ncbi.nlm.nih.gov/taxonomy/10244). The file can be donwloaded from the "Chemicals and Bioactivities" section. [Tabel 3.1](https://pubchem.ncbi.nlm.nih.gov/taxonomy/10244#section=Tested-Compounds&fullscreen=true)

In [ ]:
mpox_pchem = pd.read_csv(
    'data/pubchem/TaxID_10244_bioactivity.csv'
)
mpox_pchem = mpox_pchem['cid']
mpox_pchem = set(mpox_pchem)

In [ ]:
pchem2chembl_list = cid2chembl(mpox_pchem)

#### Checking overlap between the ChEMBL and Pubchem information

In [ ]:
mpox_chembl.update(set(pchem2chembl_list))
len(mpox_chembl)

In [ ]:
mpox_chembl

### Get proteins, mech of action, disease, and activity data from ChEMBL

In [ ]:
chembl2mech = RetMech(list(mpox_chembl))

In [ ]:
chembl2mech

In [ ]:
chembl2dis = RetDrugInd(mpox_chembl)


In [ ]:
chembl2dis

In [ ]:
with open("data/export/test.json", "w") as outfile:
    json.dump(chembl2dis, outfile, indent=4, sort_keys=False)

In [ ]:
chembl2act = RetAct(mpox_chembl)

In [ ]:
chembl2act

In [ ]:
prtn_as_chembl = Ret_chembl_protein(chembl2act) + Ret_chembl_protein(chembl2mech)
prtn_as_chembl = set(prtn_as_chembl)
prtn_as_chembl = list(prtn_as_chembl)
len(prtn_as_chembl)

In [ ]:
chembl2uprot = chembl2uniprot(prtn_as_chembl)

### Updating ChEMBL protein with gene symbols

In [ ]:
chembl2act = chembl2gene2path(chembl2uprot, chembl2act)
chembl2mech = chembl2gene2path(chembl2uprot, chembl2mech)

### Adding information to KG

In [ ]:
mpox_graph = pybel.BELGraph(name='Monkeypox Graph', version="0.0.1")

In [ ]:
mpox_graph = chem2moa_rel(chembl2mech, 'HGNC', mpox_graph)
mpox_graph = chem2dis_rel(chembl2dis, mpox_graph)
mpox_graph = chem2act_rel(chembl2act, 'HGNC', mpox_graph)
mpox_graph = gene2path_rel(chembl2uprot, 'HGNC', mpox_graph)

### Saving cache of KG here

In [ ]:
pybel.dump(mpox_graph, 'data/graph/monkeypox_basic.bel.pickle')

In [ ]:
infile=  open('data/graph/monkeypox_basic.bel.pickle','rb')
mpox_graph= pickle.load(infile)
infile.close()

In [ ]:
to_jupyter(mpox_graph)

### Extracting data from UniProt

The data was collected from the [UniProt website](https://www.uniprot.org/uniprotkb?dir=ascend&facets=reviewed:true&query=monkeypox&sort=organism_name). To ensure that the proteins are curated, we restrict the filtering to "Reviewed Swiss-Prot".

Additionally, it can be seen that the above filter results in proteins from other viruses as well. We manually selected the 11 monkeypox virus proteins and downloaded the data for the same.

In [ ]:
#Fetch all proteins from the graph
chemblProt = Ret_uprotid(chembl2uprot)
chemblProt 

In [ ]:
#chemblProt = ['P20839', 'P23526', 'O43865', 'Q8JXU8', 'P08684']

In [ ]:
# mpox_prot_df = pd.read_excel(
#     'data/uniprot/uniprot-taxonomy Monkeypox+virus+[10244] -filtered-reviewed yes.xlsx',
# )

#updated protein file from uniprot. H
mpox_prot_df = pd.read_excel(
    'data/uniprot/uniprotkb_taxonomy_id_10244_AND_reviewe_2023_08_18.xlsx',
)

mpox_prot = list(mpox_prot_df['Entry'])
mpox_prot

# Extracting data from DISEASES database from JensenLab

The data for monkeypox virus can be found [here](https://diseases.jensenlab.org/Entity?order=textmining,knowledge,experiments&textmining=10&knowledge=10&experiments=10&type1=-26&type2=9606&id1=DOID:3292). We downloaded information about the genes associated with this virus by downloading the "Text mining" filtered data only ([availabe here](https://diseases.jensenlab.org/Downloads)).

In [ ]:
colnames = 'Gene HGNC doid disease z-score confidence url'.split(' ')
mpox_human = pd.read_csv('data/jensen/human_disease_textmining_filtered_aug23.tsv',
                         sep='\t',names=colnames,header=None)
mpox_human = mpox_human.loc[mpox_human['doid'] == ('DOID:3292')]
mpox_human = mpox_human.reset_index(drop=True)
mpox_human.head(5)


In [ ]:
mpox_human.to_csv('data/jensen/mpox_human.csv')

# Conversion of DISEASES genes to UniProt ids using UniProt DB

In [ ]:
mpox_human_uprot = pd.read_csv('data/jensen/jenslab_aug23.tsv',sep='\t')
mpox_human_uprot.head(5)

In [ ]:
mpox_human_uprot

In [ ]:
mpox_genes = list(mpox_human_uprot['Entry'])
len(mpox_genes)

In [ ]:
#combining human genes from KG and DISEASES
uprots = mpox_genes+chemblProt

# Fetch info about biological processes, molecular functions and diseases from UniProt

In [ ]:
uprots_ext = ExtractFromUniProt(uprots)

In [ ]:
mpox_prot_ext = ExtractFromUniProt(mpox_prot)

In [ ]:
 
mpox_graph = uniprot_rel(mpox_prot_ext, 'MPXV', mpox_graph)

In [ ]:
to_jupyter(mpox_graph)

### Saving cache of KG here

In [ ]:
pybel.dump(mpox_graph,'data/graph/monkeypox_gene_enriched.bel.pickle')

### Chemicals targetting proteins present in graph from ChEMBL and PubChem

In [ ]:
# Since this run can be time-consuming, we store a cache file and re-use that.
# To update the information, delete the cache file
#Please scroll down to 
mpox_prot2chem = target_list_to_chemical(uprots)

In [ ]:
# filename = ('data/chembl/prot2chembl.pkl')
# outfile = open(filename,'wb')
# pickle.dump(mpox_prot2chem,outfile)

In [ ]:
#combine all proteins
uprots_list = uprots+mpox_prot
#uprots_list = uprots_list[::-1]


In [ ]:
uprot2chem = target_list_to_chemical(uprots_list)

In [ ]:
# filename = ('data/chembl/uprot2chembl.pkl')
# outfile = open(filename,'wb')
# pickle.dump(uprot2chem,outfile)

In [ ]:
uprot2chem.to_csv(f'data/chemical_protein_data_Aug23.tsv', sep='\t', index=False)

# Cached file for Chemicals targetting proteins present in graph from ChEMBL and PubChem¶

In [ ]:
infile = open('data/chembl//uprot2chembl','rb')
uprot2chem = pickle.load(infile)
infile.close()

### Filtering chemicals

Here we filter out chemicals that are active and do not have a corresponding name (such chemicals will be still at the research level within ChEMBL).

In [ ]:
active = uprot2chem.loc[uprot2chem['activity'].str.contains('activator',na=False)]
active = active.reset_index(drop=True)
active = active[active['compound_name'] != '']
active = active.reset_index(drop=True)

In [ ]:
active

In [ ]:
chem = active['chembl_id']

# Repeat steps of fetching proteins, mech of action, disease, and activity data from ChEMBL for new chemicals

In [ ]:
chem2dis = RetDrugInd(chem)

In [ ]:
chem2mech = RetMech(chem)

In [ ]:
chem2act = RetAct(chem)

In [ ]:
chembl2actmechdis = {}
chembl2actmechdis['activity'] = chem2act
chembl2actmechdis['mechanism'] = chem2mech
chembl2actmechdis['disease']= chem2dis

In [ ]:
filename = 'data/chembl/chembl2actmechdis_aug23'
outfile = open(filename,'wb')
pickle.dump(chembl2actmechdis,outfile)

In [ ]:
infile = open('data/chembl/chembl2actmechdis_aug23','rb')
chembl2actmechdis = pickle.load(infile)
infile.close()

In [ ]:
chem2act = chembl2actmechdis['activity']
chem2mech = chembl2actmechdis['mechanism']
chem2dis = chembl2actmechdis['disease']

In [ ]:
chemid2 = Ret_chembl_protein(chem2act) + Ret_chembl_protein(chem2mech)


In [ ]:
chem2uni = chembl2uniprot(chemid2)

In [ ]:
chem2uni

In [ ]:
protList = Ret_uprotid(chem2uni)
len(protList)

In [ ]:
protList_ext = ExtractFromUniProt(protList)

In [ ]:
filename = 'data/chembl/allprot_fromchembl_23'
outfile = open(filename,'wb')
pickle.dump(protList_ext,outfile)

In [ ]:
infile = open('data/chembl/allprot_fromchembl_23','rb')
protList_ext = pickle.load(infile)
infile.close()

In [ ]:
#mpox_graph = pybel.BELGraph(name='Monkeypox Graph')

In [ ]:
protList_ext

# Update KG with new nodes and relationships

In [ ]:
 
pchem_act_new= chembl2gene2path(chem2uni,chem2act)
pchem_mech_new = chembl2gene2path(chem2uni,chem2mech)
mpox_graph = chem2act_rel(pchem_act_new,'HGNC',mpox_graph)
mpox_graph = chem2moa_rel(pchem_mech_new,'HGNC',mpox_graph)
mpox_graph = chem2dis_rel(chem2dis,mpox_graph)
mpox_graph = gene2path_rel(chem2uni,'HGNC',mpox_graph)
#to_jupyter(mpox_graph)

In [ ]:
# filename = 'data/export/Monkeypox_KG_aug23.pkl'
# outfile = open(filename,'wb')
# pickle.dump(mpox_graph,outfile)

In [ ]:
infile = open('data/export/Monkeypox_KG_aug23.pkl','rb')
mpox_graph = pickle.load(infile)
infile.close()

### Annotating information from OpenTargets

To reproduce this code, you will need to download the [drug side effect](http://ftp.ebi.ac.uk/pub/databases/opentargets/platform/22.06/output/etl/parquet/fda/significantAdverseDrugReactions/) data and [target data](http://ftp.ebi.ac.uk/pub/databases/opentargets/platform/22.06/output/etl/parquet/targets/) data files.

In [ ]:
chemblid = getNodeList('ChEMBL',mpox_graph)

In [ ]:
mpox_graph = chembl2rxn_rel(chemblid,mpox_graph)

# Annotate proteins with ChEMBL URLs

In [ ]:
mpox_graph = chembl_annotation(mpox_graph)

In [ ]:
# filename = 'data/export/Monkeypox_KG_aug23.pkl'
# outfile = open(filename,'wb')
# pickle.dump(mpox_graph,outfile)
# outfile.close()

# This is the final KG for monkeypox. Execute following cells to generate statistics and plots about the KG. To generate a small section of the KG, jump to cell 'Create sub-graph for Supplementary File Outline no. 3' and execute and execute next couple of cells

In [ ]:
infile = open('data/export/Monkeypox_KG_aug23.pkl','rb')
mpox_graph = pickle.load(infile)
infile.close()

In [ ]:
mpox_graph.summarize

In [ ]:
mpox_graph.summarize

# Generation of plots for KG statistics

# Bar plots for node types

In [ ]:
node_data = {'Node':['Pathology','BiologicalProcess','Abundance','Protein'],
       'Number':[4220, 2356, 2181, 360]}
node = pd.DataFrame(node_data)
a = sns.barplot(x="Node", y="Number", data=node)
a.set(xlabel='Node',ylabel='Number',title= 'KG nodes in numbers (Total=9117)')

# Bar plots for namespace types

In [ ]:
nspace_data = {'Namespace':['ChEMBLAssay','ChEMBL','GOBP','GOMF','HGNC','MPXV','Disease','SideEffect','Reactome','MOA'],
       'Number':[1732,449,846,568,360,10,905,3315,810,132]}
nspace = pd.DataFrame(nspace_data)

#a = sns.barplot(x="Number", y="Namespace", data=nspace_data)
#a.set(xlabel='Number',ylabel='Namespace',title= 'KG Namespace in numbers')

In [ ]:
plt.figure()

a = sns.barplot(x="Number", y="Namespace", data=nspace_data)
a.set(xlabel='Number',ylabel='Namespace',title= 'KG Namespace in numbers')

plt.tight_layout()
#plt.savefig('data/export/test2.png',dpi=600)
plt.show()

In [ ]:

fig = a.get_figure()
plt.tight_layout()
plt.savefig('data/export/test.png',dpi=600)

# Load file from Open Targets for info about 'druggability'

In [ ]:
infile = open('data/opentargets//targets.pkl','rb')
targetability = pickle.load(infile)
infile.close()

In [ ]:
#Fetch human proteins from KG
protein = []
for node in mpox_graph.nodes():
    if isinstance(node,pybel.dsl.Protein):
        if node.namespace == 'HGNC':
            protein.append(node.name)
            
len(protein)

In [ ]:
targetability = targetability[targetability['approvedSymbol'].isin(protein)]
targetability = targetability.reset_index(drop=True)

In [ ]:
#targetability[targetability['approvedSymbol']=='C1R']
targetability.dropna(subset=['tractability'],inplace=True)
targetability = targetability.reset_index(drop=True)
targetability.head(5)


# Create a file with 'druggability' label yes or no for human proteins

In [ ]:
gene = []
flag = []
i = 0
#j = 0
for i in range(len(targetability)):

        
    #if pd.isna(targetability['tractability'][i].any() != True): 
    if targetability['tractability'][i][7]['value'] == True:

        gene.append(targetability['approvedSymbol'][i])
        flag.append('Yes')
        
    else:
        #print(targetability['approvedSymbol'][i])
        gene.append(targetability['approvedSymbol'][i])
        flag.append('No')

        

In [ ]:
druggability = pd.DataFrame()
druggability['Gene'] = gene
druggability['Druggable Family'] = flag
druggability.to_csv('data/opentargets/druggability.csv',sep=',')

In [ ]:
druggability

# Create sub-graph for Supplementary File Outline no. 3

In [ ]:
def filter_graph(mainGraph, vprotList):
    nsp_list = []
    chem_list = []
    for u, v, data in mainGraph.edges(data=True):
        if u.name in vprotList or v.name in vprotList:
            #print(u.name)
            nsp_list.append(u)
            #print(u)
            #print(v.name)
            nsp_list.append(v)

    nsp_graph = mainGraph.subgraph(nsp_list)
    #nsp_graph = pybel.struct.mutation.induction_expansion.get_subgraph_by_second_neighbors(mpox_graph, nsp_list, filter_pathologies=False)
    return(nsp_graph)


In [ ]:
query = 'p28 TK B4R Q8V4Y0 MKRN3 MKRN4P MKRN1 MKRN2 RNF8 TK1 SLFN12 SLFN12L CA3 CA13 CA5A CA1 CA2'.split(' ')
#query = ['Virus Diseases','Smallpox']
#query = 'NS5B CYP3A4 IMPDH1'.split(' ')

In [ ]:
query_graph = filter_graph(mpox_graph,query)

In [ ]:
to_jupyter(query_graph)

# Export final KG to other standard formats

In [ ]:
#to cytoscape compatible graphml 
pybel.to_graphml(mpox_graph,'data/export//Monkeypox_KG_jan15.graphml')

#to regular BEL format
pybel.dump(mpox_graph,'data/export//Monkeypox_KG_jan15.bel')

#to neo4j
pybel.to_csv(mpox_graph,'data/export//Monkeypox_KG_jan15.csv')

#to sif
pybel.to_sif(mpox_graph,'data/export//Monkeypox_KG_jan15.sif')

#to xml

#pybel.to

# Exporting graph to Neo4J

In [ ]:
## to Neo4j
# Exporting graph to Neo4J
import py2neo
node_map = {}

# NEO4J_USER = 'yojana'
# NEO4J_PASS = 'abc'

#neo_connection = py2neo.Graph('bolt://localhost:7687', auth=('neo4j', 'itmp'),name='mpoxkg')
neo_connection = py2neo.Graph('neo4j+s://a0760dff.databases.neo4j.io', auth=('fraunhofer', 'monkeypox-2022'))
#neo_connection.delete_all()
tx = neo_connection.begin()

nodes = list(mpox_graph)

dbio = nx.get_node_attributes(mpox_graph,'3Dbio')
uprot = nx.get_node_attributes(mpox_graph,'UniProt')
chembl = nx.get_node_attributes(mpox_graph,'ChEMBL')

for node in tqdm(nodes, desc="nodes"):
    attrs = {"namespace": node.namespace}

    if node.name and node.identifier:
        attrs["name"] = node.name
        attrs["identifier"] = node.identifier
    elif node.identifier and not node.name:
        attrs["name"] = node.identifier
    elif node.name and not node.identifier:
        attrs["name"] = node.name
    
    if node in dbio:
        attrs['3Dbio'] = dbio[node]
        
    if node in uprot:
        attrs['UniProt'] = uprot[node]
        
    if node in chembl:
        attrs['ChEMBL'] = chembl[node]

    node_map[node] = py2neo.Node(node.function, **attrs)

    tx.create(node_map[node])

edges = mpox_graph.edges(keys=True, data=True)

for u, v, key, node in tqdm(edges, desc="edges"):
    rel_type = node['relation']

    d = node.copy()
    del d['relation']

    attrs = {}

    annotations = d.pop('annotations', None)
    if annotations:
        for annotation, values in annotations.items():
            attrs[annotation] = list(values)

    citation = d.pop('citation', None)
    if citation:
        attrs['citation'] = citation.curie

    if 'evidence' in d:
        attrs['evidence'] = d['evidence']

    rel = py2neo.Relationship(node_map[u], rel_type, node_map[v], key=key, **attrs)
    tx.create(rel)
    
tx.commit()

In [ ]:
###new

In [ ]:
#Fetch human/viral proteins from KG
#use HGNC or MPXV
protein = []
for node in mpox_graph.nodes():
    if isinstance(node,pybel.dsl.Protein):
        if node.namespace == 'MPXV':
            protein.append(node.name)
            
len(protein)

In [ ]:
protein

# Implementation for Results section in manuscript

# Get list of drugs that are used to treat "Virus Diseases"

In [ ]:
query = ['Virus Diseases','Smallpox']

In [ ]:
viral_drugs_kg = filter_graph(mpox_graph,query)
to_jupyter(viral_drugs_kg)

In [ ]:
def getChemfromKG(mainGraph):

    chem_list = []
    for u, v, data in mainGraph.edges(data=True):
        
        if 'CHEMBL' in u.name:
            if u.name not in chem_list:
                chem_list.append(u.name)

        if 'CHEMBL' in v.name:
            if v.name not in chem_list:
                chem_list.append(v.name)
                
    return(chem_list)



In [ ]:
viral_drugs = getChemfromKG(viral_drugs_kg)
viral_drugs

# Function to retrieve chemicals in Phase IV, Modification to RetDrugInd function

In [ ]:
def RetDrugInd_phase4(chemblIDs) -> dict:
    """Function to retrieve associated diseases from ChEMBL

    :param chemblIDs:
    :return:
    """
    getDrugInd = new_client.drug_indication

    drugIndList = []
    for chemblid in tqdm(chemblIDs, desc='Retrieving diseases from ChEMBL'):
        drugInd = getDrugInd.filter(
            molecule_chembl_id=chemblid
        ).only(['mesh_heading','max_phase_for_ind'])
        
        data = []
        
#         print(drugInd)
#         break
        
        for ind in drugInd:
            if int(ind.get('max_phase_for_ind')<4):
                continue
            data.append(ind)
                
        
        drugIndList.append(list(data))

    named_drugIndList = dict(zip(chemblIDs, drugIndList))
    named_drugIndList = {
        k: v
        for k, v in named_drugIndList.items()
        if v
    }
    return named_drugIndList

In [ ]:
phase4_drugs = RetDrugInd_phase4(viral_drugs)
phase4_drugs


In [ ]:
#convert dict to a dataframe

#step 1
#generate list for chemicals that have sub-dict
chem_list = []
for chem in phase4_drugs:
    for dis in phase4_drugs[chem]:
        chem_list.append(chem)

#step 2
#create df from sub-dict        
phase4drugs = pd.concat([pd.DataFrame(d) for d in phase4_drugs.values()], ignore_index=True)

#step 3
#append step 1 to step 2
phase4drugs['Drug'] = chem_list

In [ ]:
phase4drugs.head(5)
phase4drugs.to_csv('data/export/phase4drugs.csv')


In [ ]:
drugs_CT4 = list(phase4_drugs.keys())
drugs_CT4

# Extend chemical nodes with Mech. of action and target proteins, filter out diseases, sideEffect and assays

In [ ]:
query_graph = filter_graph(mpox_graph,drugs_CT4)
filter_se = []
for node in query_graph:
    if node.namespace == 'SideEffect' or node.namespace == 'Disease' or node.namespace== 'ChEMBLAssay':
        filter_se.append(node)




In [ ]:
G = query_graph.copy()
G.remove_nodes_from([n for n in G if n in set(filter_se)])

In [ ]:
to_jupyter(G)

In [ ]:
mpox_chembl

In [ ]:
viral_drugs

In [ ]:
list(set(mpox_chembl)&set(viral_drugs))

# Misc

In [ ]:
def RetMech(chemblIds) -> dict:
    """Function to retrieve mechanism of actions and target proteins from ChEMBL

    :param chemblIds:
    :return:
    """
    getMech = new_client.mechanism

    mechList = []
    for chemblid in tqdm(chemblIds, desc='Retrieving mechanisms from ChEMBL'):
        mechs = getMech.filter(
            molecule_chembl_id=chemblid
        ).only(['mechanism_of_action', 'mechanism_refs','target_chembl_id','action_type'])
        
        
        mechList.append(list(mechs))
        


    named_mechList = dict(zip(chemblIds, mechList))
    named_mechList = {
        k: v
        for k, v in named_mechList.items()
        if v
    }
    return named_mechList


In [ ]:
a ='CHEMBL1643'
test = RetMech(viral_drugs)
test


In [ ]:
to_jupyter(filter_graph(mpox_graph,'NS5B'))

In [ ]:
#test
import networkx as nx

tg = pybel.BELGraph(name='Monkeypox Graph', version="0.0.1")

tg = uniprot_rel(protList_ext, 'HGNC', tg)
